In [1]:
from ome_zarr.io import parse_url
import zarr

from os.path import join

import mobie
from mobie.metadata.project_metadata import create_project_metadata, add_dataset
from mobie.metadata.dataset_metadata import create_dataset_structure, create_dataset_metadata

from faim_hcs.mobie import add_wells_to_project

from os.path import exists
import shutil

# MoBIE Project

MoBIE is a Fiji viewer to visualize large data like ome-zarr HCS plates. A MoBIE project can hold multiple datasets and each dataset consists of views of different image sources, which can be linked to tabular data. For more information check out the [MoBIE Tutorials]().

Citation: [Pape, Constantin, et al. "MoBIE: a Fiji plugin for sharing and exploration of multi-modal cloud-hosted big image data." Nature Methods (2023): 1-2.](https://www.nature.com/articles/s41592-023-01776-4)

In [376]:
# The MoBIE project folder and name of the plate/dataset
mobie_project_folder = "./mobie-example-project"

In [377]:
if exists(mobie_project_folder):
    # Remove if it already exists.
    shutil.rmtree(mobie_project_folder)

# Create a new empty MoBIE project
create_project_metadata(mobie_project_folder)

## Add 3D OME-Zarr

In [378]:
dataset_name = "Projection-Mix"

In [379]:
# Create a new dataset
create_dataset_structure(mobie_project_folder, dataset_name, file_formats=["ome.zarr"])

'./mobie-example-project/Projection-Mix'

In [380]:
# Fill in dataset metadata
create_dataset_metadata(
    dataset_folder=join(mobie_project_folder, dataset_name), 
    description="A 3D projection mix example dataset.",
    is2d=False,
)

In [381]:
# Update MoBIE project by adding the new dataset.
add_dataset(mobie_project_folder, dataset_name, is_default=True)

In [382]:
# OME-Zarr plate we want to add to a MoBIE project
plate_path = "/tungstenfs/scratch/gmicro_hcs/gmicro/eglijan/zarr-files/20230129-pilot plate 1-240123C-DMSO.zarr"
store = parse_url(plate_path, mode="r").store
    
plate = zarr.group(store=store)

In [383]:
from mobie.metadata.view_metadata import get_grid_view

In [384]:
from mobie.metadata import (
    add_regions_to_dataset,
    add_source_to_dataset,
    get_default_view,
)
from faim_hcs.UIntHistogram import UIntHistogram
from faim_hcs.mobie import hex_to_rgba

In [385]:
from mobie.view_utils import create_grid_view
from tqdm.notebook import tqdm

In [386]:
from copy import copy

In [387]:
sources = {}
# source_transforms = {}
# display_settings = {}
plate_hists = {}

well_group = "0"
dataset_folder = join(mobie_project_folder, dataset_name)

# Add wells as individual sources
wells = []
for i, row in enumerate(tqdm(list(plate.group_keys()))):
    for j, col in enumerate(tqdm(list(plate[row].group_keys()), leave=False)):
        attrs = plate[row][col][well_group].attrs.asdict()
        wells.append(row + col.zfill(2))
        path = join(plate.store.path, row, col, well_group)

        hists = [
            UIntHistogram.load(join(path, h_path)) for h_path in attrs["histograms"]
        ]
        
        group_name = f"{row}{col.zfill(2)}"
        scale = attrs["multiscales"][0]["datasets"][0]["coordinateTransformations"][0][
            "scale"
        ]
        y_spacing, x_spacing = scale[-2], scale[-1]

        for k, ch in enumerate(attrs["omero"]["channels"]):
            key = f"{ch['wavelength_id']}_{ch['label']}"

            name = f"{group_name}_{key}"
            view = get_default_view(
                source_type="image",
                source_name=name,
                menu_name="Wells",
                color=hex_to_rgba(ch["color"]),
                contrastLimits=[hists[k].quantile(0.01), hists[k].quantile(0.99)],
                opacity=1.0,
                sources=[
                    name,
                ],
            )

            add_source_to_dataset(
                dataset_folder=dataset_folder,
                source_type="image",
                source_name=name,
                image_metadata_path=path,
                file_format="ome.zarr",
                channel=k,
                view=view,
            )
            
            if key not in sources.keys():
                sources[key] = [name]
            else:
                sources[key].append(name)
                
            if key not in plate_hists.keys():
                plate_hists[key] = copy(hists[k])
            else:
                plate_hists[key].combine(hists[k])

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

...

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

In [388]:
import string
def to_position(well_name):
    r,c = well_name[0], well_name[1:]
    r = string.ascii_uppercase.index(r)
    c = int(c) - 1
    return [c, r]

In [389]:
list(sources.keys())

['C01_empty', 'C02_DAPI_20', 'C03_FITC', 'C04_Cy5']

In [390]:
from mobie.metadata import add_view_to_dataset

In [391]:
default = {
    "isExclusive": True,
    "sourceDisplays": [],
    "sourceTransforms": [],
    "uiSelectionGroup": "bookmarks"
}
for ch in sources.keys():
    view = get_grid_view(
        dataset_folder=join(mobie_project_folder, dataset_name),
        name=ch,
        sources=[[src] for src in sources[ch]],
        menu_name="Channels",
        positions=[to_position(src[:3]) for src in sources[ch]],
        use_transformed_grid=False,
    )
    view['isExclusive'] = False
    view['sourceDisplays'][0]['imageDisplay']['contrastLimits'] = [plate_hists[ch].quantile(0.01),
                                                                   plate_hists[ch].quantile(0.99)]
    add_view_to_dataset(
        dataset_folder=join(mobie_project_folder, dataset_name),
        view_name=ch,
        view=view,
        overwrite=True
    )
    
    default['sourceDisplays'].extend(view['sourceDisplays'])
    default['sourceTransforms'].extend(view['sourceTransforms'])

In [392]:
from mobie.metadata import create_region_display, add_regions_to_dataset

In [393]:
well_table = pd.DataFrame(
    {
        "region_id": wells,
        "treatment": [
            "Unknown",
        ] * len(wells)
    }
)

In [394]:
well_table

,region_id,treatment
0,A01,Unknown
1,A10,Unknown
2,A11,Unknown
3,A12,Unknown
4,A13,Unknown
...,...,...
379,P05,Unknown
380,P06,Unknown
381,P07,Unknown
382,P08,Unknown


In [395]:
add_regions_to_dataset(
    join(mobie_project_folder, dataset_name), 
    "wells",
    well_table,
)

In [396]:
region_sources = {}
for channel_wells in sources.values():
    
    for well in channel_wells:
        if well[:3] not in region_sources.keys():
            region_sources[well[:3]] = [well]
        else:
            region_sources[well[:3]].append(well)
            

In [397]:
default['sourceDisplays'].append(
    {
        "regionDisplay": {
                "opacity": 0.5,
                "lut": "glasbey",
                "name": "Wells",
                "sources": region_sources,
                "tableSource": "wells",
                "visible": True,
                "showAsBoundaries": True,
            }
    }
)

In [398]:
from mobie.view_utils import create_view, combine_views

In [399]:
from mobie.metadata.dataset_metadata import add_view_to_dataset, add_default_location_to_dataset

In [400]:
add_view_to_dataset(
    dataset_folder=join(mobie_project_folder, dataset_name),
    view_name="default",
    view=default,
)

In [401]:
# Validate MoBIE project integrety
mobie.validation.validate_project(mobie_project_folder)

Check views for dataset Projection-Mix: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1541/1541 [00:22<00:00, 69.38it/s]
Check view files for dataset Projection-Mix: 0it [00:00, ?it/s]

The project at ./mobie-example-project is a valid MoBIE project.


# Add projections